# HW09: Transformers

Remember that these homework work as a completion grade. **You can skip one section of this homework.**

In [1]:
import pandas as pd
import nltk
df = pd.read_csv('train.csv')

df.columns = ["label", "title", "lead"]
label_map = {1:"world", 2:"sport", 3:"business", 4:"sci/tech"}
def replace_label(x):
	return label_map[x]
df["label"] = df["label"].apply(replace_label) 
df["text"] = df["title"] + " " + df["lead"]
df = df.sample(n=10000) # # only use 10K datapoints
df.head()

,label,title,lead,text
20620,sci/tech,RFID tags: The people say no,"The issue has united readers of all stripes, f...",RFID tags: The people say no The issue has uni...
66604,business,Computer Associates Results Top View (Reuters),Reuters - Computer Associates International Lt...,Computer Associates Results Top View (Reuters)...
116589,business,Brazil corruption charges urged,More than 90 people in Brazil face possible ch...,Brazil corruption charges urged More than 90 p...
115889,sport,Compiled by Page 3,With real life news reading funnier than made-...,Compiled by Page 3 With real life news reading...
78872,world,"Arafat condition improves, illness still a mys...",PARIS Palestinian officials say a medical repo...,"Arafat condition improves, illness still a mys..."


## Hugginface Transformers

In [2]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig
import tensorflow as tf

config = DistilBertConfig.from_pretrained('distilbert-base-uncased')
config.num_labels = 4
transformer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=config)



/home/felix/miniconda3/envs/nlp_lss/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-12 15:21:06.566067: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-12 15:21:06.566133: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-12 15:21:10.399576: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 15:21:10.399632: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303

In [3]:
##TODO build a transformer model to do sequence classification with the goal to predict the label from the text

seq_len = 256 # use masking for padded sequences of length 256
input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
input_masks_ids = tf.keras.layers.Input(shape=(seq_len,), name='attention_mask', dtype='int32')

# build model to handle input tokens and masks
X = transformer_model(input_ids, input_masks_ids)
model = tf.keras.Model(inputs=[input_ids, input_masks_ids], outputs = X)

In [4]:
##TODO print the summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 tf_distil_bert_for_sequence_cl  TFSequenceClassifie  66956548   ['input_ids[0][0]',              
 assification (TFDistilBertForS  rOutput(loss=None,               'attention_mask[0][0]']         
 equenceClassification)         logits=(None, 4),                                                 
                                 hidden_states=None                                           

In [5]:
##TODO compile the model
model.compile(loss='sparse_categorical_crossentropy', # cost function
              optimizer='adam', # use adam as the optimizer
              metrics=['accuracy']) # compute accuracy, for scoring

**Hint:** All the vectorized pieces of text must have the same length (which will be equal to the input size). You have two options to ensure this:

1. Set the maximum length equal to the length of the shortest vectorized text
2. Choose the maximum length and then exclude all the data points that have vectors shorter than that length

**Hint:** Tensorflow requires your labels to be integers, not strings

In [6]:
##TODO split the sample into a training and a test set 
from transformers import DistilBertTokenizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X_train, X_test, y_train, y_test = train_test_split(df['text'].tolist(), df['label'].tolist(), test_size=.2)

labelEncoder = LabelEncoder().fit(df['label'])
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

##TODO prepare the dataset for tensorflow.
def prepare_ds(X, y):
    # tokenize dataset and get integer labels
    X_tf = tokenizer(X, return_tensors="tf", padding=True, truncation=True, max_length=seq_len)
    y_tf = labelEncoder.transform(y)

    # build batched tensorflow dataset
    return tf.data.Dataset.from_tensor_slices((dict(X_tf), y_tf)).batch(8)

train_ds = prepare_ds(X_train, y_train)

In [7]:
##TODO fit the model and print the obtained accuracy
model.fit(train_ds, epochs=1, verbose=2)

In [ ]:
# evaluate model on test split
test_ds = prepare_ds(X_test, y_test)
model.evaluate(test_ds)

# LSTMs

In [ ]:
from keras.layers import LSTM

##TODO create a sequential model with an embedding layer, a LSTM layer and two hidden layers with ReLu activation function, followed by dropout

In [ ]:
##TODO compile the model and fit it to predict the business label